# Exploring OpenAI API

## Task 1: Import Modules and Packages

In [11]:
from openai import OpenAI
import pandas as pd
import requests
from datetime import datetime
from pprint import pprint
import tiktoken
from pypdf import PdfReader
from IPython.display import Image, Markdown, display
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## Task 2: Set the API Key

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Task 2.1: Set Pandas Options

In [12]:
pd.set_option('display.max_colwidth', None)


def pp(df):
    return display(df.style.set_properties(subset=['emails'], **{'text-align': 'left', 'white-space': 'pre-wrap', 'width': '900px'}))

## Task X: Create Variables

In [13]:
# Path to the 'reviews' folder
reviews_folder = "./reviews"

# List of reviews
reviews = [
    "Nice socks, great colors, just enough support for wearing with a good pair of sneakers.",
    "Love Deborah Harness's Trilogy! Didn't want the story to end and hope they turn this trilogy into a movie. I would love it if she wrote more books to continue this story!!!",
    "SO much quieter than other compressors. VERY quick as well. You will not regret this purchase.",
    "Shirt a bit too long, with heavy hem, which inhibits turning over. I cut off the bottom two inches all around, and am now somewhat comfortable. Overall, material is a bit too heavy for my liking.",
    "The quality on these speakers is insanely good and doesn't sound muddy when adjusting bass. Very happy with these.",
    "Beautiful watch face. The band looks nice all around. The links do make that squeaky cheapo noise when you swing it back and forth on your wrist which can be embarrassing in front of watch enthusiasts. However, to the naked eye from afar, you can't tell the links are cheap or folded because it is well polished and brushed and the folds are pretty tight for the most part. love the new member of my collection and it looks great. I've had it for about a week and so far it has kept good time despite day 1 which is typical of a new mechanical watch."
]

## Task X. Create the reviews .txt files if not already present

In [14]:
# Create the review files in the 'emails' folder with error handling
for i, review in enumerate(reviews, start=1):
    file_path = os.path.join(reviews_folder, f"review{i}.txt")
    try:
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(review)
        print(f"✅ Successfully created: {file_path}")
    except OSError as e:
        print(f"❌ Error writing to {file_path}: {e}")

print(
    f"\nAll review files have been created in the '{reviews_folder}' folder.")

✅ Successfully created: ./reviews\review1.txt
✅ Successfully created: ./reviews\review2.txt
✅ Successfully created: ./reviews\review3.txt
✅ Successfully created: ./reviews\review4.txt
✅ Successfully created: ./reviews\review5.txt
✅ Successfully created: ./reviews\review6.txt

All review files have been created in the './reviews' folder.


## Task x: Read the Reviews from the .txt files

In [15]:
# Ensure the folder exists before reading files
if not os.path.exists(reviews_folder):
    raise FileNotFoundError(
        f"The folder '{reviews_folder}' does not exist. Please create it and add review files.")

# Read all .txt files from the 'reviews' folder
reviews = []
for filename in sorted(os.listdir(reviews_folder)):
    if filename.endswith(".txt"):
        file_path = os.path.join(reviews_folder, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            reviews.append(file.read().strip())

# Load reviews into a DataFrame
columns = ['reviews', 'emails']
df = pd.DataFrame(columns=columns)
df['reviews'] = reviews

# Display the DataFrame
df.head()

,reviews,emails
0,"Nice socks, great colors, just enough support for wearing with a good pair of sneakers.",NaN
1,Love Deborah Harness's Trilogy! Didn't want the story to end and hope they turn this trilogy into a movie. I would love it if she wrote more books to continue this story!!!,NaN
2,SO much quieter than other compressors. VERY quick as well. You will not regret this purchase.,NaN
3,"Shirt a bit too long, with heavy hem, which inhibits turning over. I cut off the bottom two inches all around, and am now somewhat comfortable. Overall, material is a bit too heavy for my liking.",NaN
4,The quality on these speakers is insanely good and doesn't sound muddy when adjusting bass. Very happy with these.,NaN


## Task x: Generate Emails for Reviews

In [17]:
chat = [{"role": "system",
         "content": "You are a polite customer support representative."}]

postfix = (
    "\n\nWrite a professional email responding to customer reviews.\n"
    "- Thank customers for positive feedback.\n"
    "- Acknowledge and address concerns professionally.\n"
    "- Encourage continued engagement with the brand.\n"
    "- Keep responses brief and highly professional.\n"
    "- Do not offer promotions, discounts, or recommend other products."
)


def getMail(review):
    chat_history = chat.copy()
    chat_history.append({"role": "user", "content": review+postfix})

    reply = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=chat_history
    )

    return reply.choices[0].message.content


df['emails'] = df.apply(lambda x: getMail(x.reviews), axis=1)
pp(df)

,reviews,emails
0,"Nice socks, great colors, just enough support for wearing with a good pair of sneakers.","Subject: Thank You for Your Feedback Dear [Customer], Thank you for taking the time to leave a review about our socks. We are thrilled to hear that you are pleased with the colors and support they provide, and that they work well with your sneakers. We want to address any concerns you may have. If there is anything that we can do to improve your experience with our products, please do not hesitate to reach out to us. We appreciate your engagement with our brand and hope to continue providing high-quality products for your satisfaction. Thank you again for your feedback. Best regards, [Your Name] Customer Support Team"
1,Love Deborah Harness's Trilogy! Didn't want the story to end and hope they turn this trilogy into a movie. I would love it if she wrote more books to continue this story!!!,"Subject: Thank You for Your Feedback Dear [Customer], Thank you for sharing your positive feedback about Deborah Harkness's Trilogy. We're delighted to hear that you enjoyed the story and would like to see it turned into a movie. Your support means a lot to us. While we appreciate your enthusiasm for the series, we currently do not have plans for a movie adaptation or continuation of the story. However, we encourage you to stay engaged with the author's work and our brand for any future updates. Thank you once again for sharing your thoughts. We value your input and are grateful for your support. Best regards, [Your Name] Customer Support Team"
2,SO much quieter than other compressors. VERY quick as well. You will not regret this purchase.,"Subject: Thank You for Your Feedback Dear valued customer, Thank you for taking the time to leave a positive review for our compressor. We are pleased to hear that you found it to be quieter and quicker than other models. Your satisfaction is very important to us. We take customer feedback seriously and strive to continuously improve our products. If you have any concerns or suggestions, please feel free to reach out to our customer support team. We are here to assist you. We appreciate your support and encourage you to stay engaged with our brand. Your feedback is invaluable in helping us maintain the quality of our products and service. Thank you once again for choosing our compressor. Sincerely, [Your Name] Customer Support Team"
3,"Shirt a bit too long, with heavy hem, which inhibits turning over. I cut off the bottom two inches all around, and am now somewhat comfortable. Overall, material is a bit too heavy for my liking.","Dear [Customer], Thank you for taking the time to share your feedback with us. We're glad to hear that you are somewhat comfortable with the adjustments made to the shirt. We have noted your concerns about the heavy material and the hem inhibiting turning over, and we will certainly take this into consideration for future product development. We appreciate your support and hope you continue to engage with our brand. If you have any further feedback or questions, please don't hesitate to reach out. Thank you again for your valuable input. Sincerely, [Your Name] Customer Support"
4,The quality on these speakers is insanely good and doesn't sound muddy when adjusting bass. Very happy with these.,"Subject: Thank You for Your Feedback Dear [Customer], Thank you for taking the time to leave a positive review of our speakers. We are thrilled to hear that you are happy with the quality and performance. Your feedback is appreciated and valued by our team. We are committed to delivering the best sound experience, and we are delighted that you have found our speakers to meet your expectations. If you have any further feedback or questions, please feel free to reach out. We look forward to serving you again in the future. Thank you for choosing our brand. Warm regards, [Your Name] Customer Support Team"
5,"Beautiful watch face. The band looks nice all around. The links do make that s